# Worksheet 08

Name:  
UID: 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.813711939291719, 5.094328419997168, 4.938581631169785, 5.5413735515095075, 5.779493679880243, 5.994679161661686, 3.743403109560141, 4.3455426487305315, 4.996928873549383, 3.215671884113352]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
s2 = np.random.normal(8, 1,10 ).tolist()
print(s2)

[8.358608926901418, 8.68028104836865, 7.947369384707107, 6.802895114667908, 8.09909979364897, 6.878581391456931, 9.48290874791877, 8.176397377171288, 9.490027277296223, 8.435082738640341]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[3.215671884113352, 4.996928873549383, 4.3455426487305315, 8.435082738640341, 9.490027277296223, 8.176397377171288, 3.743403109560141, 5.994679161661686, 9.48290874791877, 5.779493679880243]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

For each mixture, there was 3 parameters. We have two mixtures which make total of 6 parameters.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[8.435082738640341, 9.490027277296223, 8.176397377171288, 9.48290874791877]
[3.215671884113352, 4.996928873549383, 4.3455426487305315, 3.743403109560141, 5.994679161661686, 5.779493679880243]
P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 8.896104035256656,  mean_2 = 4.679286559582557
var_1 = 0.3569007244658635,  var_2 = 1.0285067988476535


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0]*prob_s[0]/prob_x)
    prob_s1_x.append( pdf_i[1]*prob_s[1]/prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  3.215671884113352
probability of observing that point if it came from cluster 0 =  1.0985612453161562e-55
probability of observing that point if it came from cluster 1 =  0.1409176555966155
point =  4.996928873549383
probability of observing that point if it came from cluster 0 =  1.349447661609496e-26
probability of observing that point if it came from cluster 1 =  0.36982064714056917
point =  4.3455426487305315
probability of observing that point if it came from cluster 0 =  5.586853793180065e-36
probability of observing that point if it came from cluster 1 =  0.36799179632403495
point =  8.435082738640341
probability of observing that point if it came from cluster 0 =  0.4853276379875392
probability of observing that point if it came from cluster 1 =  0.0004932494299424987
point =  9.490027277296223
probability of observing that point if it came from cluster 0 =  0.27991271663578127
probability of observing that point if it came from cluster 1 =  6.8854802622569485e-06
poin


1. **Point 3.215671884113352:**
   - Assigned Cluster: 1

2. **Point 4.996928873549383:**
   - Assigned Cluster: 1

3. **Point 4.3455426487305315:**
   - Assigned Cluster: 1

4. **Point 8.435082738640341:**
   - Assigned Cluster: 0

5. **Point 9.490027277296223:**
   - Assigned Cluster: 0

6. **Point 8.176397377171288:**
   - Assigned Cluster: 0

7. **Point 3.743403109560141:**
   - Assigned Cluster: 1

8. **Point 5.994679161661686:**
   - Assigned Cluster: 1

9. **Point 9.48290874791877:**
   - Assigned Cluster: 0

10. **Point 5.779493679880243:**
    - Assigned Cluster: 1

Now, let's compare these assignments to the truth:

- **True Assignments:**
  - Points 1, 2, 3, 7, and 10 belong to Cluster 1.
  - Points 4, 5, 6, 8, and 9 belong to Cluster 0.

- **Estimated Assignments:**
  - Points 1, 2, 3, 7, and 10 are assigned to Cluster 1.
  - Points 4, 5, 6, 8, and 9 are assigned to Cluster 0.

The estimated assignments seem to align with the truth based on the higher probabilities. 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s1_x) ]
var = [ sum([x[0] * (x[1]-mean[0])**2 for x in zip(prob_s0_x, data)]) , sum([x[0] * (x[1]-mean[0])**2 for x in zip(prob_s1_x, data)]) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.4,  P(S_2) = 0.6
mean_1 = 8.89845817186888,  mean_2 = 5.898468861530469
var_1 = 1.4209513777044163,  var_2 = 112.98616904461836


g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append(pdf_i[0]*prob_s[0]/prob_x)
    prob_s1_x.append( pdf_i[1]*prob_s[1]/prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  3.215671884113352
probability of observing that point if it came from cluster 0 =  9.445412477829268e-05
probability of observing that point if it came from cluster 1 =  0.003529899613383215
point =  4.996928873549383
probability of observing that point if it came from cluster 0 =  0.006475405505378449
probability of observing that point if it came from cluster 1 =  0.0035307824320487146
point =  4.3455426487305315
probability of observing that point if it came from cluster 0 =  0.0016557594009468943
probability of observing that point if it came from cluster 1 =  0.003530561340357049
point =  8.435082738640341
probability of observing that point if it came from cluster 0 =  0.26621886663356437
probability of observing that point if it came from cluster 1 =  0.003530005102324569
point =  9.490027277296223
probability of observing that point if it came from cluster 0 =  0.25745098308257114
probability of observing that point if it came from cluster 1 =  0.0035291113845756983
po

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)


1. **Point 3.215671884113352:**
   - Assigned Cluster: 1

2. **Point 4.996928873549383:**
   - Assigned Cluster: 0

3. **Point 4.3455426487305315:**
   - Assigned Cluster: 1

4. **Point 8.435082738640341:**
   - Assigned Cluster: 0

5. **Point 9.490027277296223:**
   - Assigned Cluster: 0

6. **Point 8.176397377171288:**
   - Assigned Cluster: 0

7. **Point 3.743403109560141:**
   - Assigned Cluster: 1

8. **Point 5.994679161661686:**
   - Assigned Cluster: 0

9. **Point 9.48290874791877:**
   - Assigned Cluster: 0

10. **Point 5.779493679880243:**
    - Assigned Cluster: 0
